In [140]:
import pandas as pd
import numpy as np
import ast 
import re

In [141]:
metadata = pd.read_csv('metadata.csv')

In [142]:
metadata.iloc[165]['id'] = 'd2007100901'
metadata.iloc[164]['id'] = 'd2007100902'

In [143]:
ids = metadata['id']

In [144]:
import ast 
import re

# we assume a valid name is of the format 'ABC DEF', which is upper and has at least two parts.
def isName(string):
    if string.isupper() and ' ' in string and (',' not in string):
        return True
    else:
        return False
    
def elementwise_check(name, debator_name):
    for i in name:
        if i not in debator_name:
            return False
    return True

# if it is a debator name or host name
def isDebator(name, debators):
    if isName(name) and sum([elementwise_check(name, debator) for debator in debators])==1:
        return True
    else:
        return False            
    
def isHost(name, host_name):
    return isName(name) and (name in host_name)  

for iid in ids:
    debators = ast.literal_eval(metadata[metadata['id'] == iid]['for'].tolist()[0]) + ast.literal_eval(metadata[metadata['id'] == iid]['against'].tolist()[0])
    debators = [x.upper() for x in debators]
    transcript_df = pd.DataFrame(columns=debators)
    transcript = open('transcripts/transcripts_' + iid).read()
    transcript = re.sub("\[TS\].{8}\[TE\]", ' ', transcript)
#     if iid = 'd2008'
    transcript_cleaned = re.sub("\[(.*?)\]", '', transcript)[1:]
    
    ## we assume the host name to be the first name appears
    host_name = ''
    
    
    for word in transcript_cleaned.split():
        if word.isupper():
            if host_name == '':
                host_name += word
            else:
                host_name = host_name + ' ' + word
        else:
            break
            
            
    name = ''
    temp_name = ''
    script = []
    output = []
    
    data = transcript_cleaned.split()
    round_num = ''
    for i,word in enumerate(data):
        
        if word.lower() == 'round' and (('1' in data[i+1]) or ('one' in data[i+1].lower())):
            round_num = 1
        if word.lower() == 'round' and (('2' in data[i+1]) or ('two' in data[i+1].lower())):
            round_num = 2
        if word.lower() == 'round' and (('3' in data[i+1]) or ('three' in data[i+1].lower())):
            round_num = 3
        
        if word.isupper():
            if temp_name == '':
                temp_name += word
            else:
                temp_name = temp_name + ' ' + word
        else:
            ## if temp_name is empty, it indicates that we are still appending script
            if temp_name == '':
                script.append(word)

            elif isName(temp_name):
                ## if the name of the last speaker is debator, we store his name and script.
                if isDebator(name, debators):
                    ## get the full name of the debator
                    debator_full_name = pd.Series(debators)[[elementwise_check(name, d) for d in debators]].iloc[0]
                    output.append([debator_full_name, ' '.join(script), round_num])
                ## if the name of the last speaker is host, we do not store his name and script.

                name = temp_name
                temp_name = ''
                script = [word]
                end = False
            else:
                ## if somehow the word is mistakenly taken as temp_name due to its upper case, we continue appending it.
                script.append(temp_name)
                script.append(word)## = script + " " + temp_name + " "+word
                temp_name = ''

    output_df = pd.DataFrame(output, columns = ['Speaker', 'Script', 'Round'])
    
    ## find the index of the first entry in Round that is 1 (ther may be edge cases that the host is simply introducing each section.)
    try:
        first_one_index = output_df.Round[output_df.Round == 1].index[0]
        output_df.loc[:first_one_index,'Round'] = ''
        output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)
    except:
        output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)
    

## Notes: 
2007100901 style of name is "Abc Def:"

20081007 the website got the debator name wrong.

20100209 and after uses the name sytle: "Abc Def:"

In [145]:
def isName(string):
    word_list = string.split()
    if len(word_list) == 1:
        return word_list[-1] == ':'
    else:
        return word_list[-1][-1] == ':'


def isDebator(name, debators):
    if name == '':
        return False
    # remove ':' when checking if it is in debators list.
    if isName(name) and sum([elementwise_check(name[:-1], debator) for debator in debators])==1:
        return True
    else:
        return False  

In [146]:
target_position = ids[ids == 'd20100209'].index.values[0]
filtered_id = ids[:target_position+1]

In [147]:
filtered_id = filtered_id.append(pd.Series(['d2007100901'])).reset_index(drop = True)

In [148]:
for iid in filtered_id:    
    debators = ast.literal_eval(metadata[metadata['id'] == iid]['for'].tolist()[0]) + ast.literal_eval(metadata[metadata['id'] == iid]['against'].tolist()[0])
    #debators = [x.upper() for x in debators]
    #debators[1] = 'Rashid Khalidi'
    transcript_df = pd.DataFrame(columns=debators)
    transcript = open('transcripts/transcripts_' + iid).read()
    transcript = re.sub("\[TS\].{8}\[TE\]", ' ', transcript)
    #     if iid = 'd2008'
    transcript_cleaned = re.sub("\[(.*?)\]", '', transcript)[1:]


    name = ''
    temp_name = ''
    script = []
    output = []
    # Since the first word of the speaker also begins with upper case, we need to notate this situation.
    end = False
    
    round_num = ''
    data = transcript_cleaned.split()


    for i,word in enumerate(data):
        
        if word.lower() == 'round' and (('1' in data[i+1]) or ('one' in data[i+1].lower())):
            round_num = 1
        if word.lower() == 'round' and (('2' in data[i+1]) or ('two' in data[i+1].lower())):
            round_num = 2
        if word.lower() == 'round' and (('3' in data[i+1]) or ('three' in data[i+1].lower())):
            round_num = 3
            
            
        if word[0].isupper() and (not "." in word) and not end:
            if temp_name == '':
                temp_name += word
            else:
                temp_name = temp_name + ' ' + word

            if word[-1] == ':':
                end = True

        else:
            ## if temp_name is empty, it indicates that we are still appending script
            if temp_name == '':
                script.append(word)
                end = False
            elif isName(temp_name):
                #print(temp_name)
                ## if the name of the last speaker is debator, we store his name and script.
                if isDebator(name, debators):
                    ## get the full name of the debator
                    debator_full_name = pd.Series(debators)[[elementwise_check(name[:-1], d) for d in debators]].iloc[0]
                    output.append([debator_full_name, ' '.join(script), round_num])
                ## if the name of the last speaker is host, we do not store his name and script.

                name = temp_name
                temp_name = ''
                script = [word]
                end = False
            else:
                ## if somehow the word is mistakenly taken as temp_name due to its upper case, we continue appending it.
                script.append(temp_name)
                script.append(word)## = script + " " + temp_name + " "+word
                end = False
                temp_name = ''

    output_df = pd.DataFrame(output, columns = ['Speaker', 'Script', 'Round'])
    try:
        first_one_index = output_df.Round[output_df.Round == 1].index[0]
        output_df.loc[:first_one_index,'Round'] = ''
        output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)
    except:
        output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)



In [149]:
debators

['Arthur Laffer', 'Phil Gramm', 'Nouriel Roubini', 'Laura Tyson']

## example_id = d20100209 (this is should be treated specially as it has spelling error of its debator.)

In [150]:
def elementwise_check(name, debator_name):
    for i in name:
        if i not in debator_name:
            return False
    return True

In [151]:
iid = 'd20100209'
debators = ast.literal_eval(metadata[metadata['id'] == iid]['for'].tolist()[0]) + ast.literal_eval(metadata[metadata['id'] == iid]['against'].tolist()[0])
#debators = [x.upper() for x in debators]
debators[1] = 'Rashid Khalidi'
transcript_df = pd.DataFrame(columns=debators)
transcript = open('transcripts/transcripts_' + iid).read()
transcript = re.sub("\[TS\].{8}\[TE\]", ' ', transcript)
transcript_cleaned = re.sub("\[(.*?)\]", '', transcript)[1:]


name = ''
temp_name = ''
script = []
output = []
# Since the first word of the speaker also begins with upper case, we need to notate this situation.
end = False

round_num = ''
data = transcript_cleaned.split()
for i,word in enumerate(data):
    
    if word.lower() == 'round' and (('1' in data[i+1]) or ('one' in data[i+1].lower())):
        round_num = 1
    if word.lower() == 'round' and (('2' in data[i+1]) or ('two' in data[i+1].lower())):
        round_num = 2
    if word.lower() == 'round' and (('3' in data[i+1]) or ('three' in data[i+1].lower())):
        round_num = 3
    
    if word[0].isupper() and (not "." in word) and not end:
        if temp_name == '':
            temp_name += word
        else:
            temp_name = temp_name + ' ' + word
            
        if word[-1] == ':':
            end = True
            
    else:
        ## if temp_name is empty, it indicates that we are still appending script
        if temp_name == '':
            script.append(word)
            end = False
        elif isName(temp_name):
            ## if the name of the last speaker is debator, we store his name and script.
            if isDebator(name, debators):
                ## get the full name of the debator
                debator_full_name = pd.Series(debators)[[elementwise_check(name[:-1], d) for d in debators]].iloc[0]
                output.append([debator_full_name, ' '.join(script), round_num])
            ## if the name of the last speaker is host, we do not store his name and script.

            name = temp_name
            temp_name = ''
            script = [word]
            end = False
        else:
            ## if somehow the word is mistakenly taken as temp_name due to its upper case, we continue appending it.
            script.append(temp_name)
            script.append(word)## = script + " " + temp_name + " "+word
            end = False
            temp_name = ''

output_df = pd.DataFrame(output, columns = ['Speaker', 'Script', 'Round'])

try:
    first_one_index = output_df.Round[output_df.Round == 1].index[0]
    output_df.loc[:first_one_index,'Round'] = ''
    output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)
except:
    output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)


In [152]:
output_df

,Speaker,Script,Round
0,Roger Cohen,His answer was - he suggested maybe Einstein. ...,
1,Roger Cohen,"Good evening, ladies and gentlemen. In life, w...",1
2,Roger Cohen,The United States can step back while ensuring...,1
3,Stuart Eizenstat,"Thank you, John. I couldn't disagree more with...",1
4,Stuart Eizenstat,The only bipartisan foreign policy which we ha...,1
5,Rashid Khalidi,Yes.,1
6,Rashid Khalidi,"Good evening, ladies and gentlemen. Thank you ...",1
7,Itamar Rabinovich,Thank you. I urge you strongly to vote against...,1
8,Stuart Eizenstat,"Well, what would happen is that the United Sta...",2
9,Stuart Eizenstat,It would very harmful for Israel because Israe...,2


##  Test
example id = d20061129


In [129]:
## example id = d20061129
import ast 
debators = ast.literal_eval(metadata[metadata['id'] == 'd20061129']['for'].tolist()[0]) + ast.literal_eval(metadata[metadata['id'] == 'd20061129']['against'].tolist()[0])
debators = ast.literal_eval(metadata[metadata['id'] == 'd20070314']['for'].tolist()[0]) + ast.literal_eval(metadata[metadata['id'] == 'd20061129']['against'].tolist()[0])

debators = [x.upper() for x in debators]
transcript_df = pd.DataFrame(columns=debators)

In [130]:
transcript_df

,MICHAEL CRICHTON,RICHARD S. LINDZEN,PHILIP STOTT,STANLEY L. COHEN,MOHAMMAD-MAHMOUD OULD MOHAMEDOU,MARK PERRY


In [131]:
transcript = open('transcripts/transcripts_' + 'd20061129').read()
transcript = open('transcripts/transcripts_' + 'd20070314').read()

In [132]:
import re
transcript = re.sub("\[TS\].{8}\[TE\]", ' ', transcript)
transcript_cleaned = re.sub("\[(.*?)\]", '', transcript)[1:]

In [133]:
debators

['MICHAEL CRICHTON',
 'RICHARD S. LINDZEN',
 'PHILIP STOTT',
 'STANLEY L. COHEN',
 'MOHAMMAD-MAHMOUD OULD MOHAMEDOU',
 'MARK PERRY']

In [134]:
transcript_df

,MICHAEL CRICHTON,RICHARD S. LINDZEN,PHILIP STOTT,STANLEY L. COHEN,MOHAMMAD-MAHMOUD OULD MOHAMEDOU,MARK PERRY


In [135]:
# we assume a valid name is of the format 'ABC DEF', which is upper and has at least two parts.
def isName(string):
    if string.isupper() and ' ' in string and (',' not in string):
        return True
    else:
        return False
    
# if it is a debator name or host name
def isDebator(name, debators):
    if isName(name) and sum([name in debator for debator in debators])==1:
        return True
    else:
        return False            
    
## we assume the host name to be the first name appears
host_name = ''
for word in transcript_cleaned.split():
    if word.isupper():
        if host_name == '':
            host_name += word
        else:
            host_name = host_name + ' ' + word
    else:
        break
        
def isHost(name, host_name):
    return isName(name) and (name in host_name)   

In [136]:
name = ''
temp_name = ''
script = []
output = []
data = transcript_cleaned.split()
round_num = ''
for i,word in enumerate(data):
    if word.lower() == 'round' and (('1' in data[i+1]) or ('one' in data[i+1].lower())):
        round_num = 1
    if word.lower() == 'round' and (('2' in data[i+1]) or ('two' in data[i+1].lower())):
        round_num = 2
    if word.lower() == 'round' and (('3' in data[i+1]) or ('three' in data[i+1].lower())):
        round_num = 3
    
        
    if word.isupper():
        if temp_name == '':
            temp_name += word
        else:
            temp_name = temp_name + ' ' + word
    else:
        ## if temp_name is empty, it indicates that we are still appending script
        if temp_name == '':
            script.append(word)

        elif isName(temp_name):
            ## if the name of the last speaker is debator, we store his name and script.
            if isDebator(name, debators):
                ## get the full name of the debator
                debator_full_name = pd.Series(debators)[[name in d for d in debators]].iloc[0]
                output.append([debator_full_name, ' '.join(script), round_num])
            ## if the name of the last speaker is host, we do not store his name and script.
                
            name = temp_name
            temp_name = ''
            script = [word]
        else:
            ## if somehow the word is mistakenly taken as temp_name due to its upper case, we continue appending it.
            script.append(temp_name)
            script.append(word)## = script + " " + temp_name + " "+word
            temp_name = ''
            

In [137]:
output_Df = pd.DataFrame(output, columns = ['Speaker', 'Script', 'Round'])

In [138]:
try:
    first_one_index = output_df.Round[output_df.Round == 1].index[0]
    output_df.loc[:first_one_index,'Round'] = ''
    output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)
except:
    output_df.to_csv('cleaned_transcript_data/cleaned_transcript_'+ iid + '.csv', header = True)

In [139]:
output_Df

,Speaker,Script,Round
0,RICHARD S. LINDZEN,"Okay, I’d like to thank Intelligence Squared, ...",
1,RICHARD S. LINDZEN,Okay.,
2,MICHAEL CRICHTON,The microphone goes up. Before I begin I want ...,
3,MICHAEL CRICHTON,This doesn’t need to happen. We’re allowing it...,
4,PHILIP STOTT,Brian may I just take one second to thank very...,
5,PHILIP STOTT,…..environmental concerns. But let me end with...,
6,PHILIP STOTT,….he was mad.,
7,RICHARD S. LINDZEN,"Yeah, of course it could be. That’s, uh, goes ...",
8,RICHARD S. LINDZEN,It may be—,
9,RICHARD S. LINDZEN,What I was referring to was the issue of varia...,


In [174]:
output_Df.to_csv('cleaned_transcript_data/test'+'.csv', header = True)